In [ ]:
"""
    This notebook combines the Precipitation Frequency Estimates (PFE) for Multi-stations into one big csv file.
    The headers and footnotes are trimmed off
    
    contact
    ----------
    Dr. KENNETH EKPETERE |kenneth.ekpetere@gmail.com

    """

In [1]:
import pandas as pd
import requests
import time
import os
import csv
from io import StringIO
import urllib3

#### **Combine pfe into one file**

In [ ]:

# Define the folder path
# folder_path = "out_trim_pfe_test"
folder_path = "out_trim_pfe_files"

# Define the output CSV file
# output_file = "output_atlas14_test.csv"
output_file = "output_atlas14.csv"

# List to store processed data from all files
all_data = []

# Loop through all files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".csv"):
        # Split the filename to extract ID, latitude, and longitude
        file_parts = file_name.split('_')
        if len(file_parts) >= 5:  # Ensure the filename format is as expected
            ID = file_parts[2]
            
            # Construct full file path
            file_path = os.path.join(folder_path, file_name)
            
            # Read the CSV file, skipping the first row (header row starts from row 2)
            df = pd.read_csv(file_path, skiprows=1)
            
            # Set column headers to non-numeric
            df.columns = df.columns.astype(str)
            
            # Ensure 'by duration for ARI (years):' is in columns
            if 'by duration for ARI (years):' in df.columns:
                
                # Extract the 'by duration for ARI (years):' column
                duration_labels = df['by duration for ARI (years):'].values
                
                # Reshape the data
                for col in df.columns[1:]:  # Skip 'by duration for ARI (years):'
                    ari_value = col  # The ARI value from the header
                    values = df[col].values
                    
                    # Create a dictionary for the current file with one unique row per ARI
                    data = {'ID': ID, 'ARI': ari_value}
                    
                    # Map the duration labels to their corresponding values
                    data.update(dict(zip(duration_labels, values)))
                    
                    # Append the reshaped data to the list
                    all_data.append(data)

# Combine all data into one DataFrame
final_df = pd.DataFrame(all_data)

# Rename the columns to the desired format
final_df.columns = ['ID', 'ARI', '5-min', '10-min', '15-min', '30-min', '60-min', '2-hr', '3-hr', '6-hr', 
                    '12-hr', '24-hr', '2-day', '3-day', '4-day', '7-day', '10-day', '20-day', '30-day', 
                    '45-day', '60-day']

# Ensure that ID and ARI are treated as numeric values for proper sorting
final_df['ID'] = pd.to_numeric(final_df['ID'], errors='coerce')
final_df['ARI'] = pd.to_numeric(final_df['ARI'], errors='coerce')

# Sort by ID first, then by ARI in ascending order
final_df = final_df.sort_values(by=['ID', 'ARI'], ascending=True)

# Save the final DataFrame to a CSV file
final_df.to_csv(output_file, index=False)

print(f"Output saved to {output_file}")


#### **Run if empty files exist or breaks**

In [2]:

# folder_path = "out_trim_pfe_test"
folder_path = "out_trim_pfe_files"

# Define the output CSV file
# output_file = "output_atlas14_test.csv"
output_file = "output_atlas14.csv"

# List to store processed data from all files
all_data = []

# Loop through all files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".csv"):
        # Split the filename to extract ID, latitude, and longitude
        file_parts = file_name.split('_')
        if len(file_parts) >= 5:  # Ensure the filename format is as expected
            ID = file_parts[2]
            
            # Construct full file path
            file_path = os.path.join(folder_path, file_name)
            
            # Check if the file is non-empty before reading
            if os.path.getsize(file_path) > 0:
                try:
                    # Read the CSV file, skipping the first row (header row starts from row 2)
                    df = pd.read_csv(file_path, skiprows=1)
                    
                    # Set column headers to non-numeric
                    df.columns = df.columns.astype(str)
                    
                    # Ensure 'by duration for ARI (years):' is in columns
                    if 'by duration for ARI (years):' in df.columns:
                        
                        # Extract the 'by duration for ARI (years):' column
                        duration_labels = df['by duration for ARI (years):'].values
                        
                        # Reshape the data
                        for col in df.columns[1:]:  # Skip 'by duration for ARI (years):'
                            ari_value = col  # The ARI value from the header
                            values = df[col].values
                            
                            # Create a dictionary for the current file with one unique row per ARI
                            data = {'ID': ID, 'ARI': ari_value}
                            
                            # Map the duration labels to their corresponding values
                            data.update(dict(zip(duration_labels, values)))
                            
                            # Append the reshaped data to the list
                            all_data.append(data)
                except pd.errors.EmptyDataError:
                    print(f"Skipping empty file: {file_name}")
            else:
                print(f"Skipping empty or corrupted file: {file_name}")

# Combine all data into one DataFrame
if all_data:
    final_df = pd.DataFrame(all_data)

    # Rename the columns to the desired format
    final_df.columns = ['ID', 'ARI', '5-min', '10-min', '15-min', '30-min', '60-min', '2-hr', '3-hr', '6-hr', 
                        '12-hr', '24-hr', '2-day', '3-day', '4-day', '7-day', '10-day', '20-day', '30-day', 
                        '45-day', '60-day']

    # Ensure that ID and ARI are treated as numeric values for proper sorting
    final_df['ID'] = pd.to_numeric(final_df['ID'], errors='coerce')
    final_df['ARI'] = pd.to_numeric(final_df['ARI'], errors='coerce')

    # Sort by ID first, then by ARI in ascending order
    final_df = final_df.sort_values(by=['ID', 'ARI'], ascending=True)

    # Save the final DataFrame to a CSV file
    final_df.to_csv(output_file, index=False)

    print(f"Output saved to {output_file}")
else:
    print("No valid data found to process.")


Output saved to output_atlas14.csv
